# CS-5600/6600 Lecture 9 - Cleaning, Transforming, and Preparing Data

**Instructor: Dylan Zwick**

*Weber State University*

Reference: [Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/) by Aurélien Géron - [End-to-end Machine Learning Project](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb)

Today we'll look at methods for cleaning, transforming, and preparing data. We'll also look a bit into hyperparameter optimization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer

from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer

from sklearn.metrics import root_mean_squared_error

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV, cross_val_score

## Data Exploration

The data that we'll be using for this lecture is a California housing dataset. The data consists of publicly available information for "block groups" from the 1990 California census. So, it's old data (don't get excited by the amazing low prices), but we can still analyze it.

It's saved as a .csv file on my Google Drive, and we can import it as a Pandas dataframe with the code below.

In [ ]:
#Import the California housing data
url = 'https://drive.google.com/uc?export=download&id=1V1Dw4dg7HYpPa7x2MzcUvzNb4ixVgxeL'
housing = pd.read_csv(url)
housing.head()

So, for each housing block group we know:
* The longitude and latitude
* The median age of the houses
* The total number of rooms (note this is the total number of rooms in the block group)
* The total number of bedrooms
* The total number of households
* The "median income" of those who live there.
* The median house value (what we're trying to predict)
* A categorical ocean_proximity

Let's see if we can get some more info about our data:

In [ ]:
housing.info()

It looks like the only feature for which we have missing data is total_bedrooms. We'll return to what we can do about this soon.

First, let's take a look at that categorical feature. How many categories are there, and what type of counts do we get for them?

In [ ]:
housing["ocean_proximity"].value_counts()

OK. Let's check out some  descriptive stats on our numeric features:

In [ ]:
housing.describe()

It looks like the median income isn't in dollars. It also looks like it might be capped at 15. Maybe we can get an even better idea of how these features are distributed through a histogram:

In [ ]:
housing.hist(bins=50, figsize=(12, 8))
plt.show()

It looks like there's a maximum cutoff for median income, median age, and median house value. This is something that might be a problem, or it might not be, depending on what we want to do with these predictions. We'll pretend here that it is not.

Now, before we go any further, let's set aside a test set.

In [ ]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

## Data Cleaning and Transformation

To figure out what variables might be useful for our model, we can first check out the correlations between each variable and the median house value.

In [ ]:
corr_matrix = housing.corr(numeric_only=True)

corr_matrix["median_house_value"].sort_values(ascending=False)

Looks like median income will be important. Not surprising.

If we want a more granular look at how some of the variables correlate with median house price, we can use plot the scatter matrix:

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
pd.plotting.scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

Now, the total number of rooms, bedrooms, or even people in a block group isn't really that interesting to us, given the number of households within a block group can vary significantly. What would probably be more valuable for our analysis is the number of rooms per house, the percentage of rooms that are bedrooms, and the average household size. We can derive these for each block group:

In [ ]:
housing["rooms_per_house"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_ratio"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["people_per_house"] = housing["population"] / housing["households"]

Checking whether these derives values correlate with the median house value:

In [ ]:
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

Let's break our data into two sets - the predictors and what we want to predict.

In [ ]:
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

Now, what can we do about those missing *total_bedroom* values?

Well, we could drop the rows with missing values. Or, we could just drop the column.

Or, we could *impute* those values with something that makes sense. Let's do that.

In [ ]:
housing_fillna = housing.copy()

median = housing["total_bedrooms"].median()
housing_fillna["total_bedrooms"].fillna(median, inplace=True)

This is a fairly unsophisticated way to do it. We can also use a class called an *imputer*.

In [ ]:
imputer = SimpleImputer(strategy="median")

We can now use the imputer to calculate the median for all numeric columns.

In [ ]:
housing_num = housing.select_dtypes(include=[np.number]) #First pick out the numeric columns.

In [ ]:
imputer.fit(housing_num); #Then calculate the medians.

In [ ]:
imputer.statistics_ #Here are the medians.

In [ ]:
housing_num.median().values #We can check.

Now, we'll use this imputer to fill in all our missing values.

In [ ]:
X = imputer.transform(housing_num)

Note the imputer comes with the names of all its columns.

In [ ]:
imputer.feature_names_in_

Now, what about that categorical variable?

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(8)

We can make it numeric with an *ordinal encoder*

In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_encoded[:8]

In [ ]:
ordinal_encoder.categories_

However, this suggests proximity relations among the categories that don't make sense. We can instead use a "one hot" encoder:

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_1hot

Usually one hot encodings are stored as a sparse matrix, which is sensible. However, for now we'll use a dense matrix.

In [ ]:
cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

Note we want to do this because we want to remember exactly what our training categories were.

In [ ]:
df_test = pd.DataFrame({"ocean_proximity": ["INLAND", "NEAR BAY"]})
pd.get_dummies(df_test)

In [ ]:
cat_encoder.transform(df_test)

In [ ]:
df_test_unknown = pd.DataFrame({"ocean_proximity": ["<2H OCEAN", "ISLAND"]})
pd.get_dummies(df_test_unknown)

In [ ]:
cat_encoder.handle_unknown = "ignore"
cat_encoder.transform(df_test_unknown)

## Data Scaling

Now, let's scale our data so everything has, more or less, the same range. We can do this with a simple MinMaxScaler:

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num)

But it probably makes sense to use a standard scalar, which shifts everything by the average and divides everything by the standard deviation.

In [ ]:
std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(housing_num)

Our population seems to follow a power law distribution:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
housing["population"].hist(ax=axs[0], bins=50)
housing["population"].apply(np.log).hist(ax=axs[1], bins=50)
axs[0].set_xlabel("Population")
axs[1].set_xlabel("Log of population")
axs[0].set_ylabel("Number of districts")
plt.show()

We can get a more useful dataset if we take its logarithm.

##Pipelines

Let's build a pipeline that combines these cleaning and scaling operations.

In [ ]:
log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)
log_pop = log_transformer.transform(housing[["population"]])

In [ ]:
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

In [ ]:
housing_num_prepared = num_pipeline.fit_transform(housing_num)
housing_num_prepared[:2].round(2)

In [ ]:
df_housing_num_prepared = pd.DataFrame(
    housing_num_prepared, columns=num_pipeline.get_feature_names_out(),
    index=housing_num.index)

Let's combine the numeric and categorical transformations.

In [ ]:
num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [ ]:
preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object)),
)

In [ ]:
housing_prepared = preprocessing.fit_transform(housing)

We can combine all our transformations into one pipeline:

In [ ]:
def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())
default_location_pipeline = make_pipeline(SimpleImputer(strategy="median"))
preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house", ratio_pipeline(), ["population", "households"]),
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
        ("location", default_location_pipeline, ["longitude", "latitude"]),
    ],
    remainder=default_num_pipeline)

Applying this transformation to our data:

In [ ]:
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared.shape

Here are our feature names.

In [ ]:
preprocessing.get_feature_names_out()

##Creating Our Model

Let's try a decision tree.

In [ ]:
tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(housing, housing_labels);

In [ ]:
housing_predictions = tree_reg.predict(housing)
tree_rmse = root_mean_squared_error(housing_labels, housing_predictions)
tree_rmse

Nailed it!

Clearly, we've overfit. How can we get around that? Cross-validation!

In [ ]:
tree_rmses = -cross_val_score(tree_reg, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)

In [ ]:
pd.Series(tree_rmses).describe()

Let's try a random forest model instead.

In [ ]:
forest_reg = make_pipeline(preprocessing,
                           RandomForestRegressor(random_state=42))
forest_rmses = -cross_val_score(forest_reg, housing, housing_labels,
                                scoring="neg_root_mean_squared_error", cv=10)

In [ ]:
pd.Series(forest_rmses).describe()

In [ ]:
forest_reg.fit(housing, housing_labels)
housing_predictions = forest_reg.predict(housing)
forest_rmse = root_mean_squared_error(housing_labels, housing_predictions)
forest_rmse

Better!

Let's see if we can find the optimal number of features for the random forest to consider.

In [ ]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42)),
])
param_grid = [
    {'random_forest__max_features': [4, 6, 8, 10]}
]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring='neg_root_mean_squared_error')
grid_search.fit(housing, housing_labels);

In [ ]:
grid_search.best_params_

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()